# Lab 4 - Model operationalization

In the previous lab, we used `Automates ML` to select the optimal model and registered the best performing version of the model in Model Registry.

Now, we are ready to deploy the model.

The model can be deployed to a variety of target runtimes, including:
- Azure Container Instance
- Azure Kubernetes Service
- IoT Edge
- FPGA


In this lab, we will deploy the model as a web service in Azure Container Instance.

![AML Arch](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/amlarch.png)

## Connect to the workspace

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()

Found the config file in: /Users/jarekk/repos/jakazmie/MTC_AzureAILabs/DataScienceTrack/01-aml-walkthrough/aml_config/config.json


## Retrieve the model
The model is registered in the workspace's Model Registry. First, download the model to your local directory.


In [3]:
from azureml.core.model import Model
import os

model=Model(ws, 'AutoML8cd542bf8best')
model.download(target_dir = '.')
 
!ls -l

total 4024
-rw-r--r--  1 jarekk  staff    14368 Nov  4 13:53 deploy.ipynb
-rw-r--r--  1 jarekk  staff  2040259 Nov  4 13:43 model.pkl


## Deploy as web service

To build the correct environment for ACI, provide the following:
* A scoring script that invokes the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

### Create scoring script

Create the scoring script, called score.py, used by the web service call to invoke the model.

You must include two required functions in the scoring script:
* The `init()` function, which loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats can be used.


In [4]:
%%writefile score.py
import json
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.applications import resnet50

from azureml.core.model import Model

def init():
    # Instantiate VGG16 featurizer
    global featurizer
    featurizer = resnet50.ResNet50(
            weights = 'imagenet', 
            input_shape=(224,224,3), 
            include_top = False,
            pooling = 'avg')

    # Load the model
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path(model_name = '<<modelid>>')
    model = joblib.load(model_path)
  

def run(raw_data):
    try:
        # convert json to numpy array
        img = np.array(json.loads(raw_data)['data'])
        # normalize as required by ResNet50
        img = resnet50.preprocess_input(img)
        # extract bottleneck features
        features = featurizer.predict(img)
        # make prediction
        predictions = model.predict(features)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps(predictions.tolist())

Writing score.py


### Create environment file
Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `sklearn`, `tensorflow`, `h5py` and `azureml-sdk`.

To ensure the fit results are consistent with the training results, the SDK dependency versions need to be the same as the environment that trains the model. 

In [5]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_pip_package("h5py")
myenv.add_pip_package("pynacl==1.2.1")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of `myenv.yml` file

In [6]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
  - h5py
  - pynacl==1.2.1
- scikit-learn



### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. The default is 1 core and 1 gigabyte of RAM. Since we are using ResNet50 featurizer we are CPU bound.  In this lab we will use the defaults but you should always go through the proper performance plannig exercise to find the right configuration.

In [7]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "aerial",  "method" : "classifier"}, 
                                               description='Predict aerial images')

### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment file (`myenv.yml`)
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

In [8]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='aerial-classifier-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image aerial-classifier-svc:1, operation "Succeeded"
Creating service
Running................................................
FailedACI service creation operation finished, operation "Failed"
Service creation failed, unexpected error response:
{'code': 'AciDeploymentFailed', 'message': 'Aci Deployment failed', 'details': [{'code': 'CrashLoopBackOff', 'message': "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance aerial-classifier-svc.\nYou can also try to run image jkamllab0754569115.azurecr.io/aerial-classifier-svc:1 locally. More information at http://aka.ms/debugimage"}]}
CPU times: user 2.66 s, sys: 216 ms, total: 2.87 s
Wall time: 9min 4s


Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [9]:
print(service.scoring_uri)

None


## Test deployed service

You will now test the deployed model with 3 test images.  

The following code goes through these steps:
1. Send the data as a JSON array to the web service hosted in ACI. 

2. Use the SDK's `run` API to invoke the service. You can also make raw calls using any HTTP tool such as curl.

3. Print the returned predictions 

 
First, download sample images.

In [ ]:
%%sh
mkdir samples
cd samples
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/barren-1.png
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/cultivated-1.png
wget -nv https://azureailabs.blob.core.windows.net/aerialsamples/developed-1.png
ls -l


Define utility function that wraps loading images and invoking the service.

In [ ]:
from PIL import Image
import numpy as np
import json

def score(pathnames, service):
    images = []
    for pathname in pathnames:
        img = Image.open(pathname)
        img = np.asarray(img).tolist()
        images.append(img)
    images = json.dumps({"data": images})
    images = bytes(images, encoding='utf8')
    results = json.loads(service.run(input_data=images))
    return results

Call the service.

In [ ]:
pathnames = ['samples/barren-1.png', 'samples/developed-1.png', 'samples/cultivated-1.png']

results = score(pathnames, service)

print(results)

## Clean up resources

To keep the resource group and workspace for other tutorials and exploration, you can delete only the ACI deployment using this API call:

In [ ]:
service.delete()